
<div style="border:1px solid black; padding:20px 20px;text-align: justify;text-justify: inter-word">
    <strong>Project - Motor Control 
    

<br/>


In [2]:
!pip install --upgrade tdmclient

In [3]:
!pip install tqdm scipy

In [4]:
!pip install numpy matplotlib pandas scipy tqdm

  Using cached pandas-2.1.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (18 kB)
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
Using cached pandas-2.1.3-cp311-cp311-macosx_11_0_arm64.whl (10.8 MB)
Using cached pytz-2023.3.post1-py2.py3-none-any.whl (502 kB)


<strong>Imports :

In [3]:
import os
import sys
import math
from statistics import mean
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

from vision import *

import cv2


In [6]:
import tdmclient.notebook
await tdmclient.notebook.start()

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
await tdmclient.notebook.stop()

In [5]:
def angle_to_rotate(pos_i, pos_f, theta_i):
    a = pos_f-pos_i
    theta_f = np.arctan2(a[1], a[0])
    r_theta = (theta_f*180)/np.pi-theta_i
    return r_theta
    
pos_thymio = np.array([1,1])
pos_goal = np.array([30,45])
theta = angle_to_rotate(pos_thymio,pos_goal,0)
print(theta)

56.61148642388849


In [6]:
def distance_to_cover(pos_i, pos_f):
    distance = np.linalg.norm(pos_f-pos_i)
    return distance

pos_thymio = np.array([0,0])
pos_goal = np.array([30,45])
theta = angle_to_rotate(pos_thymio,pos_goal,0)
dist = distance_to_cover(pos_thymio,pos_goal)
print(dist)

54.08326913195984


In [7]:
nb_node = 3
initial_orientation = 0 #degree
positions = np.array([[0,0],[30,45],[50,50],[17,15],[50,10]]);
rotations = np.zeros(positions.shape[0]-1)
distances =  np.zeros(positions.shape[0]-1)

rotations[0] = angle_to_rotate(positions[0],positions[1],initial_orientation)
orientation = initial_orientation+rotations[0]

for i in range(1,positions.shape[0]-1):
    rotations[i] = angle_to_rotate(positions[i],positions[i+1],orientation)
    distances[i-1] = distance_to_cover(positions[i-1],positions[i])
    orientation = orientation+rotations[i]

distances[distances.size-1]=distance_to_cover(positions[distances.size-1],positions[distances.size])

print('Rotations :',rotations)
print('Distances :',distances)

Rotations : [  56.30993247  -42.27368901 -147.35155915  124.6996675 ]
Distances : [54.08326913 20.61552813 48.10405388 33.37663854]


In [8]:
print(3537/121)

29.231404958677686


## Async implementation:

In [9]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 0102d6c0-0c6c-479f-8406-474092b0daba

In [10]:
await node.wait_for_variables()
node.var

{'_fwversion': [14, 0],
 '_id': [-10787],
 '_imot': [0, 0],
 '_integrator': [0, 0],
 '_productId': [8],
 '_vbat': [807, 808],
 'acc': [0, 0, 20],
 'acc._tap': [0],
 'button.backward': [0],
 'button.center': [0],
 'button.forward': [0],
 'button.left': [0],
 'button.right': [0],
 'buttons._mean': [10522, 8984, 13583, 13562, 14859],
 'buttons._noise': [31, 27, 40, 32, 40],
 'buttons._raw': [10515, 8984, 13579, 13556, 14856],
 'event.args': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'event.source': [-10787],
 'leds.bottom.left': [0, 0, 0],
 'leds.bottom.right': [0, 0, 0],
 'leds.circle': [0, 0, 0, 0, 0, 0, 0, 0],
 'leds.top': [0, 0, 0],
 'mic._mean': [254],
 'mic.intensity': [2],
 'mic.threshold': [0],
 'motor.left.pwm': [0],
 'motor.left.speed': [0],
 'motor.left.target': [0],
 'motor.right.pwm': [0],
 'motor.right.speed': [0],
 'motor.right.target': [0],
 'prox.comm.rx':

In [11]:
def angle_to_rotate(pos_i, pos_f, theta_i):
    a = pos_f-pos_i
    theta_f = np.arctan2(a[1], a[0])
    r_theta = (theta_f*180)/np.pi-theta_i
    return r_theta

def distance_to_cover(pos_i, pos_f):
    distance = np.linalg.norm(pos_f-pos_i)
    return distance

In [12]:
def path(positions, initial_orientation):
    nb_node = 3
    rotations = np.zeros(positions.shape[0]-1)
    distances =  np.zeros(positions.shape[0]-1)
    
    rotations[0] = angle_to_rotate(positions[0],positions[1],initial_orientation)
    orientation = initial_orientation+rotations[0]
    
    for i in range(1,positions.shape[0]-1):
        rotations[i] = angle_to_rotate(positions[i],positions[i+1],orientation)
        distances[i-1] = distance_to_cover(positions[i-1],positions[i])
        orientation = orientation+rotations[i]

    distances[distances.size-1]=distance_to_cover(positions[distances.size-1],positions[distances.size])
    rotation_ratios = rotations/360
    return distances, rotations, orientation, rotation_ratios


<strong>Kalman filter : 

In [13]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [14]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import numpy as np
from tqdm import tqdm

<strong> Variance for position :

In [15]:
qpx = 0.04 # variance on position state
qpy = 0.04
rpx = 0.25 # variance on position measurement 
rpy = 0.25

qpx = 0.04 # variance on position state
qpy = 0.04 
rpx = 0.25 # variance on position measurement 
rpy = 0.25


<strong>Variance for angle : 

In [16]:
std_speed = 11.77
q_theta_nu = std_speed/2 # variance on speed state
r_theta_nu = std_speed/2 # variance on speed measurement 

print(q_theta_nu,r_theta_nu)

5.885 5.885


In [17]:
# Initialising the remaining constants
# units: length [mm], time [s]
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
Q = np.array([[qpx, 0, 0], [0, qpy, 0],[0, 0, q_theta_nu]]); #w matrix
rotation_conv_factor = 0.41; #0.8 for 50 speed, 0.41 for 100 speed
speed_conv_factor_straight = 0.338; #straight
speed_conv_factor_left = 0.403;
speed_conv_factor_right = 0.425;

speed_conv_factor = 0.374 #mean of the two
transition_thresh = 500
l = 100 #mm, distance between the two wheels

def kalman_filter( x_est_prev, P_est_prev, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0,):

    ## Prediciton through the a priori estimate
    # estimated mean of the state
    if abs(Vr-Vl) < 30:
        speed_conv_factor = speed_conv_factor_straight
    elif Vr<Vl:
        speed_conv_factor = speed_conv_factor_right
    else:
        speed_conv_factor = speed_conv_factor_left
    
    
    orientation = np.double(x_est_prev[2][0]%360)*2*np.pi/360
    #x_est_prev[2][0] = orientation
    B = np.array([[Ts*np.cos(orientation)/2,Ts*np.cos(orientation)/2],
         [Ts*np.sin(orientation)/2,Ts*np.sin(orientation)/2],
         [-Ts*360/(l*2*np.pi), Ts*360/(l*2*np.pi)]])
    U = np.array([[Vr*speed_conv_factor],[Vl*speed_conv_factor]])
    x_est_a_priori = np.dot(A, x_est_prev) + np.dot(B, U);
  
    
    # Estimated covariance of the state
    P_est_a_priori = np.dot(A, np.dot(P_est_prev, A.T));
    P_est_a_priori = P_est_a_priori + Q if type(Q) != type(None) else P_est_a_priori
    
    ## Update         
    # y, C, and R for a posteriori estimate, depending on transition
    if (camera_x != 0) : #XOR (one or the other but not both)
        # transition detected aka camera provides the position
        y = np.array([[camera_x],[camera_y],[camera_angle]])
        H = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
        R = np.array([[rpx, 0, 0],[0, rpy,0],[0, 0, r_theta_nu]]) #not the value of the variance of the position measurments
        
        # innovation / measurement residual
        i = y - np.dot(H, x_est_a_priori); #measured-mean aka difference between the two states
        # measurement prediction covariance
        S = np.dot(H, np.dot(P_est_a_priori, H.T)) + R;
                 
        # Kalman gain (tells how much the predictions should be corrected based on the measurements)
        K = np.dot(P_est_a_priori, np.dot(H.T, np.linalg.inv(S)));
            
        # a posteriori estimate
        x_est = x_est_a_priori + np.dot(K,i);
        P_est = P_est_a_priori - np.dot(K,np.dot(H, P_est_a_priori));

    else:
        x_est = x_est_a_priori
        P_est = P_est_a_priori
     
    return x_est, P_est

Test of the filter

In [18]:
#test of the kalman filter
x_est = np.array([[0], [0], [0]]) # in mm !!!
#P_est = [1000 * np.ones(2)]

P_est = 0
Vr = 100;
Vl = 100;

Ts = 0.1

camera_x = 4
camera_y = 0
camera_angle = 0

#for i in range(0,20):
new_x_est, new_P_est = kalman_filter(x_est, P_est, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0);
x_est = new_x_est;
P_est = new_P_est;

print(x_est[2][0])
print("new_x_est\n",new_x_est,"\n new_P_est\n", new_P_est)

-9.438649121200193e-17
new_x_est
 [[ 3.38000000e+00]
 [ 0.00000000e+00]
 [-9.43864912e-17]] 
 new_P_est
 [[0.04  0.    0.   ]
 [0.    0.04  0.   ]
 [0.    0.    5.885]]


In [19]:
import time

In [20]:
def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

In [21]:
import asyncio
import time
    
async def drive_for_seconds(client, seconds):
    # Set the motor speeds
    v = {
        "motor.left.target": [int(99)],
        "motor.right.target": [int(100)],
    }
    await node.set_variables(v)
    await asyncio.sleep(seconds)


async def turn_for_seconds(client, seconds,right):
    # Set the motor speeds
    if seconds != 0:
        if right==1:
            v = {
                "motor.left.target": [int(100)],
                "motor.right.target": [int(-100)],
            }
        else:
            v = {
                "motor.left.target": [int(-100)],
                "motor.right.target": [int(100)],
            }
        await node.set_variables(v)
    await asyncio.sleep(abs(seconds))

In [22]:
'''
vstop = {
    "motor.left.target": [int(0)],
    "motor.right.target": [int(0)],
}
await node.set_variables(vstop)
'''

'\nvstop = {\n    "motor.left.target": [int(0)],\n    "motor.right.target": [int(0)],\n}\nawait node.set_variables(vstop)\n'

In [23]:
def filter():#x_est, P_est, Ts):
    global x_est, P_est, Ts, read, angle, Vr, Vl

    thymio_data.append({"left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]}) 

    #print(thymio_data[-1]["left_speed"])
    Vl = thymio_data[-1]["left_speed"]
    Vr = thymio_data[-1]["right_speed"]
    #print(Vr, Vl)

    x_est, P_est = kalman_filter(x_est, P_est, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0);
    #print(x_est)
    

In [24]:
node.send_set_variables(motors(0, 0))


## Global navigation

In [27]:
from globalNavigation import Point, Edge, Graph, build_graph, Astar 

In [31]:
def global_navigation(polys, start, goal):

    graph = Graph(polys)
    build_graph(graph)
    
    nodes, closed = Astar(graph, start, goal)
    
    #for node in nodes:
    nodes = nodes[::-1]
    coordinates = np.array(nodes)
    print(coordinates)
    
    points = np.zeros((len(coordinates),2))
    
    for i in range(len(coordinates)):
        points[i]= [coordinates[i].x,coordinates[i].y]
        
    print(points*2) 
    return points
'''Change the Scale !!!!'''

'Change the Scale !!!!'

## Local avoidance

In [28]:
def test_saw_obstacle(threshold, verbose=True):
    print("\t Test saw obstacle: ")
    if any([x > threshold for x in node['prox.horizontal'][0:5]]):
        if verbose: print("\t\t Obstacle detected")
        return True
    
    return False

In [29]:
async def path_following(motor_speed=100, threshold=0,  verbose=False):
    
    global n, positions, x_est, P_est, Ts, read, Vr, Vl, n

    saw_obstacle = False
    
    if verbose: print("\t Path following ")
    await node.set_variables(motors(motor_speed, motor_speed))
           
    prev_state="local"
    
    while not saw_obstacle:
        
        if test_saw_obstacle(threshold, verbose=False):
            if prev_state=="local": 
                prev_state="global"
            saw_obstacle = True
            await client.sleep(0.1) #otherwise, variables would not be updated
        else:
            start = time.time()
            if (positions[n][0]+position_threshold > x_est[0][0]) and (positions[n][0]-position_threshold < x_est[0][0]):
                if (positions[n][1]+position_threshold > x_est[1][0]) and (positions[n][1]-position_threshold < x_est[1][0]):
                    n+=1
            angle = angle_to_rotate([x_est[0][0],x_est[1][0]],positions[n],x_est[2][0]);
            if (abs(angle)<angle_threshold):
                #Vr = 100
                #Vl = 100
                drive_task = asyncio.create_task(drive_for_seconds(client, Ts))
                await drive_task
    
            else:
                if angle>0:
                    right = 1
                    #Vr = -100
                    #Vl = 100
                else:
                    right = 0
                    #Vr = 100
                    #Vl = -100

                turn_task = asyncio.create_task(turn_for_seconds(client, Ts, right))
                await turn_task
            #print(x_est)
            stop = time.time()
            #print("time elapsed : ", start - stop)
            
    return 

In [30]:
async def local_avoidance(motor_speed=100, threshold=0, verbose=False):
    obstacleSpeedGain = [6, 4, -2, -6, -8]
    saw_obstacle = True
    speedL = motor_speed
    speedR = motor_speed
    prev_state = "global"

    if verbose:
        print("Local avoidance")

    while saw_obstacle:
        if test_saw_obstacle(threshold, verbose=False):
            if prev_state == "global":
                prev_state = "local"
                print("\t Transitioning to local avoidance")

            for i in range(5):
                speedL += node["prox.horizontal"][i] * obstacleSpeedGain[i] // 100
                speedR += node["prox.horizontal"][i] * obstacleSpeedGain[4 - i] // 100

          
            await node.set_variables(motors(speedL, speedR))
            if verbose:
                print("\t\t Adjusting for obstacle")

        else:
            saw_obstacle = False
            if prev_state == "local":
                #prev_state = "global"
                print(prev_state)
                print("\t Transitioning to global navigation")
                
               

            # Reset motor speeds when no obstacles are detected
            speedL = motor_speed
            speedR = motor_speed
            await node.set_variables(motors(speedL, speedR))
            
            # Add a small delay to avoid continuously checking for obstacles too quickly
            await client.sleep(0.1)




In [ ]:
thymio_data = []
initial_orientation = 0 #degree

# Calculation of the fastest path # To get from the vision
polys = [[Point(170, 599), Point(170,731), Point(291,731), Point(291, 599)], [Point(431, 432), Point(398,489), Point(431, 550), Point(501, 550), Point(535, 491), Point(503, 435)], [Point(78, 129), Point(78,420), Point(531, 420), Point(531,129)]]
start = Point(76, 81)
goal = Point(498, 762)

positions = global_navigation(polys, start, goal)

positions = np.array([[0,0],[200,100],[10,30]])#[0,0],[30,45],[50,50],[17,15],[50,10]
angle_threshold = 5 #degrees
x_est = np.array([[positions[0][0]], [positions[0][1]], [initial_orientation]])
P_est = 0
Ts = 0.1
count = 0
read = 1
Vr = 0
Vl = 0
position_threshold = 10

n = 1
await node.wait_for_variables() # wait for Thymio variables values
rt = RepeatedTimer(Ts, filter)#, x_est, P_est, Ts)
try:
    print("entered try")
    for i in range(10):
        await path_following(verbose=True)
        await local_avoidance(verbose=True)
    #node.send_set_variables(motors(0, 0))
finally:
    rt.stop()
    node.send_set_variables(motors(0, 0))
    

[Point(76.00, 81.00) Point(78.00, 420.00) Point(498.00, 762.00)]
[[ 152.  162.]
 [ 156.  840.]
 [ 996. 1524.]]
entered try
	 Path following 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	

	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw obstacle: 
	 Test saw

In [ ]:
await node.set_variables(motors(0, 0))

## Main

In [7]:
thymio_data = []
initial_orientation = 0 #degree

## ----- alexei -----
## ----- ------ -----

# Open the video capture
cap = cv2.VideoCapture(0)
# Read a frame from the video capture
ret, frame = cap.read()

# Applying vision functions
thymio_result = locate_thymio(frame)
obstacles_result = locate_static_obstacles(frame, 0)
goal_result = locate_goal(frame)  
table_origin_result = locate_table_origin(frame)

start = Point(thymio_result[0], thymio_result[1])
goal = Point(goal_result[0], goal_result[1])
table_origin = Point(table_origin_result[0], table_origin_result[1])
polys = obstacles_result[2]
num_polys = obstacles_result[0]

## ----- ------ -----
## ----- ------ -----

# # Calculation of the fastest path # To get from the vision
# polys = [[Point(170, 599), Point(170,731), Point(291,731), Point(291, 599)], [Point(431, 432), Point(398,489), Point(431, 550), Point(501, 550), Point(535, 491), Point(503, 435)], [Point(78, 129), Point(78,420), Point(531, 420), Point(531,129)]]
# start = Point(76, 81)
# goal = Point(498, 762)

positions = global_navigation(polys, start, goal)

print(positions)

# positions = np.array([[0,0],[200,100],[10,30]])#[0,0],[30,45],[50,50],[17,15],[50,10]
angle_threshold = 5 #degrees
x_est = np.array([[positions[0][0]], [positions[0][1]], [initial_orientation]])
P_est = 0
Ts = 0.1
count = 0
read = 1
Vr = 0
Vl = 0
position_threshold = 10

n = 1
await node.wait_for_variables() # wait for Thymio variables values
rt = RepeatedTimer(Ts, filter)#, x_est, P_est, Ts)

async def main():    
     global x_est, P_est, Ts, read, Vr, Vl, n
   
    
     try:
        for i in range(0,100):
            start = time.time()
            if (positions[n][0]+position_threshold > x_est[0][0]) and (positions[n][0]-position_threshold < x_est[0][0]):
                if (positions[n][1]+position_threshold > x_est[1][0]) and (positions[n][1]-position_threshold < x_est[1][0]):
                    n+=1
            angle = angle_to_rotate([x_est[0][0],x_est[1][0]],positions[n],x_est[2][0]);
            if (abs(angle)<angle_threshold):
                #Vr = 100
                #Vl = 100
                drive_task = asyncio.create_task(drive_for_seconds(client, Ts))
                await drive_task
   
            else:
                if angle>0:
                    right = 1
                    #Vr = -100
                    #Vl = 100
                else:
                    right = 0
                    #Vr = 100
                    #Vl = -100

                turn_task = asyncio.create_task(turn_for_seconds(client, Ts, right))
                await turn_task
             #print(x_est)
            stop = time.time()
            print("time elapsed : ", start - stop)
    finally:
        rt.stop()
        node.send_set_variables(motors(0, 0))
await main()

NameError: name 'cv2' is not defined

## With vision

In [ ]:
thymio_data = []
initial_orientation = 0 #degree

# Open the video capture
cap = cv2.VideoCapture(0)
# Read a frame from the video capture
ret, frame = cap.read()

# Applying vision functions
thymio_result = locate_thymio(frame)
obstacles_result = locate_static_obstacles(frame, 0)
goal_result = locate_goal(frame)  
table_origin_result = locate_table_origin(frame)

start = Point(thymio_result[0], thymio_result[1])
goal = Point(goal_result[0], goal_result[1])
table_origin = Point(table_origin_result[0], table_origin_result[1])
polys = obstacles_result[2]
num_polys = obstacles_result[0]

# Calculation of the fastest path # To get from the vision
# Should be calculated directly with the camera feed

polys = [[Point(170, 599), Point(170,731), Point(291,731), Point(291, 599)], [Point(431, 432), Point(398,489), Point(431, 550), Point(501, 550), Point(535, 491), Point(503, 435)], [Point(78, 129), Point(78,420), Point(531, 420), Point(531,129)]]
start = Point(76, 81)
goal = Point(498, 762)

positions = global_navigation(polys, start, goal)
#positions = np.array([[0,0],[200,100],[10,30]])#[0,0],[30,45],[50,50],[17,15],[50,10]
angle_threshold = 5 #degrees
x_est = np.array([[positions[0][0]], [positions[0][1]], [initial_orientation]])
P_est = 0
Ts = 0.1
count = 0
read = 1
Vr = 0
Vl = 0
position_threshold = 10

n = 1
await node.wait_for_variables() # wait for Thymio variables values
rt = RepeatedTimer(Ts, filter)#, x_est, P_est, Ts)
rt_2 = RepeatedTimer(Ts, camera_acquisition)


try:
    print("entered try")
    for i in range(10):
        await path_following(verbose=True)
        await local_avoidance(verbose=True)
    #node.send_set_variables(motors(0, 0))
finally:
    rt.stop()
    node.send_set_variables(motors(0, 0))
    

In [ ]:
def camera_acquisition()
    global Ts, camera_picture_available
    
    
    locate_thymio

    x_est, P_est = kalman_filter(x_est, P_est, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0);
    #print(x_est)